In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:                
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text                           
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
#This line will open a new chrome window and start the scraping.

path = "C:/Users/4019/Anaconda3/envs/ds_salary_prediction/Notebooks/chromedriver"

df = get_jobs('data scientist',10, False, path, 15)
df
#df.to_csv('glassdoor_jobs.csv', index = False)

 x out worked
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,Our Networking Analytics vision is to provide ...,3.9,Amazon\n3.9,Sydney,"Seattle, WA",10000+ employees,1994,Company - Public,Internet,Information Technology,$10+ billion (AUD),"Google, Microsoft, Walmart"
1,Data Scientist - Product Analytics,-1,Atlassian is continuing to hire with all inter...,4.4,Atlassian\n4.4,Sydney,"Sydney, Australia",1001 to 5000 employees,2002,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,GitHub
2,Data Scientist,-1,Data Scientist\n\n\nSydney\n\nApply now\n\nRef...,3.8,Macquarie Group\n3.8,Sydney,"Sydney, Australia",10000+ employees,1969,Company - Public,Investment Banking & Asset Management,Finance,$10+ billion (AUD),"Goldman Sachs, Commonwealth Bank of Australia,..."
3,Data Scientist,-1,Job Expectations\n\nA fantastic opportunity at...,3.7,nbn™\n3.7,Sydney,"Sydney, Australia",1001 to 5000 employees,2009,Other Organisation,"Cable, Internet & Telephone Providers",Telecommunications,Unknown / Non-Applicable,-1
4,Data Scientist,-1,We are hiring for a Data Scientist for a Globa...,3.9,nbn™\n3.7,Sydney,"Melbourne, Australia",1 to 50 employees,2012,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
5,Data Scientist,-1,Data Scientist from a strong Product Developme...,3.0,The Network IT\n3.0,Canberra,"Denver, CO",1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
6,Data Scientist,-1,Requisition Title -\nData Scientist\n\nJob Num...,4.0,ANZ Banking Group\n4.0,Melbourne,"Melbourne, Australia",10000+ employees,1835,Company - Public,Banks & Building Societies,Finance,Unknown / Non-Applicable,-1
7,Data Scientist,-1,Data Scientists / Engineer with advanced analy...,3.8,HAYS\n3.8,Melbourne,"London, United Kingdom",5001 to 10000 employees,1968,Company - Private,-1,-1,$2 to $5 billion (AUD),"PageGroup, Robert Half, Robert Walters"
8,Data Scientist,-1,Data Analyst/Scientist required for a 6 month ...,4.1,Talent International\n4.1,Perth,"Sydney, Australia",201 to 500 employees,1995,Company - Private,Staffing & Outsourcing,Business Services,$1 to $2 billion (AUD),"Real Staffing, Aerotek"
9,Data Scientist,-1,Our Federal Government Client is seeking a ski...,4.1,Peoplebank\n4.1,Canberra,"Sydney, Australia",1001 to 5000 employees,1990,Company - Private,Staffing & Outsourcing,Business Services,$1 to $2 billion (AUD),-1
